# SuperHero API Notebook

This notebook explores the **SuperHero API**[](https://superheroapi.com/) to fetch data about superheroes and villains from various comic universes (Marvel, DC, etc.).

We will:
- Get an access token (requires GitHub login)
- Fetch character data by ID (full profile, powerstats, biography, etc.)
- Use the search endpoint for name-based lookup
- Extract and analyze powerstats, bios, and other info
- Create simple visualizations and comparisons

API Base: `https://superheroapi.com/api/<your-access-token>/`

Docs: https://superheroapi.com/
Character IDs list: https://superheroapi.com/ids.html

## 1. Setup & Imports

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

# Optional: for hiding your token
# from getpass import getpass
# Or hardcode for HW (don't commit to Git!)
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"  # Replace this!

BASE_URL = f"https://superheroapi.com/api/{ACCESS_TOKEN}"

## 2. Getting Your Access Token

1. Go to https://superheroapi.com/
2. Click "Access Token Here" (requires GitHub login)
3. Generate/copy your token
4. Paste it into `ACCESS_TOKEN` above

Test the connection:

In [ ]:
# Quick test with a known ID (Batman = 70)
test_url = f"{BASE_URL}/70"
response = requests.get(test_url)

if response.status_code == 200:
    data = response.json()
    print("API is working! Response keys:", list(data.keys()))
    pprint(data, indent=2)
else:
    print("Error:", response.status_code, response.text)

## 3. Fetching a Single Character by ID

Example IDs from https://superheroapi.com/ids.html:
- 1: A-Bomb
- 70: Batman
- 149: Captain America
- 332: Hulk
- 620: Spider-Man

In [ ]:
def get_character_by_id(char_id):
    url = f"{BASE_URL}/{char_id}"
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"Error for ID {char_id}: {resp.status_code}")
        return None

# Fetch Spider-Man (ID 620)
spiderman = get_character_by_id(620)
if spiderman:
    print("Spider-Man Data:")
    pprint(spiderman)

## 4. Extracting Specific Sections

The full /id returns everything, but you can also hit sub-endpoints like /powerstats, /biography, etc.

In [ ]:
# Powerstats only
power_url = f"{BASE_URL}/620/powerstats"
power_resp = requests.get(power_url).json()
print("Spider-Man's Powerstats:")
pprint(power_resp)

# Biography only
bio_url = f"{BASE_URL}/620/biography"
bio_resp = requests.get(bio_url).json()
print("\nSpider-Man's Biography:")
pprint(bio_resp)

## 5. Collecting Multiple Characters into a DataFrame

In [ ]:
heroes = ['70', '149', '332', '620']  # Batman, Cap, Hulk, Spidey
data_list = []

for hid in heroes:
    char = get_character_by_id(hid)
    if char and char.get('response') == 'success':
        row = {
            'id': char['id'],
            'name': char['name'],
            'full_name': char['biography'].get('full-name', 'N/A'),
            'publisher': char['biography'].get('publisher', 'N/A'),
            'alignment': char['biography'].get('alignment', 'N/A'),
            'intelligence': int(char['powerstats'].get('intelligence', 0)),
            'strength': int(char['powerstats'].get('strength', 0)),
            'speed': int(char['powerstats'].get('speed', 0)),
            'durability': int(char['powerstats'].get('durability', 0)),
            'power': int(char['powerstats'].get('power', 0)),
            'combat': int(char['powerstats'].get('combat', 0)),
            'image_url': char['image'].get('url', 'N/A')
        }
        data_list.append(row)

df = pd.DataFrame(data_list)
df

## 6. Basic Analysis & Visualization

In [ ]:
# Average powerstats
power_cols = ['intelligence', 'strength', 'speed', 'durability', 'power', 'combat']
print("Average Stats:")
print(df[power_cols].mean())

# Bar chart comparison
df_plot = df.set_index('name')[power_cols]
df_plot.plot(kind='bar', figsize=(10,6))
plt.title('Powerstats Comparison')
plt.ylabel('Score (0-100)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Bonus: Search by Name

In [ ]:
def search_hero(name):
    url = f"{BASE_URL}/search/{name}"
    resp = requests.get(url)
    if resp.status_code == 200:
        data = resp.json()
        if data['response'] == 'success':
            return data['results']
    return []

# Example: search for 'deadpool'
results = search_hero('deadpool')
if results:
    print(f"Found {len(results)} results for 'deadpool'")
    for r in results[:3]:
        print(r['id'], r['name'])

## Conclusion

The SuperHero API provides easy access to powerstats, biographies, and more for hundreds of characters. Unlike the old Marvel API, it covers multiple universes and requires only a simple token.

Next steps: Add more characters, handle errors better, explore appearance/work/connections, or build comparisons (e.g., Marvel vs DC).

Remember: Replace `YOUR_ACCESS_TOKEN_HERE` before running!